In [179]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [180]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers

In [181]:
# Define data path and class names
data_path = "/kaggle/input/pomegranate-fruits/Fruits"
class_names = ["Healthy", "Bacterial", "Fungal"]

In [182]:
# Load and preprocess images
images = []
labels = []

In [183]:
for class_index, class_name in enumerate(class_names):
    class_folder = os.path.join(data_path, class_name)
    for image_name in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (224, 224))  # ResNet-50 input size
        images.append(image)
        labels.append(class_index)

images = np.array(images)
labels = np.array(labels)
print(images.shape)
print(labels.shape)

(419, 224, 224, 3)
(419,)


In [184]:
 # Define data path and class names
data_path = "/kaggle/input/pomegranate-test/pomegranate_test"
class_names = ["test_healthy", "test_bacterial", "test_fungal"]
# Load and preprocess images
x_test2 = []
y_test2= []
for class_index, class_name in enumerate(class_names):
    class_folder = os.path.join(data_path, class_name)
    for image_name in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (224, 224))  # ResNet-50 input size
        x_test2.append(image)
        y_test2.append(class_index)

x_test2 = np.array(x_test2)
y_test2 = np.array(y_test2)
print(x_test2.shape)
print(y_test2.shape) 

(84, 224, 224, 3)
(84,)


In [185]:
# Convert labels to categorical
labels = to_categorical(labels, num_classes=3)
print(labels.shape)

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)
print(x_train.shape[0])
print(x_test.shape[0])
print(x_train.shape)
print(y_train.shape)
print(y_test.shape)

(419, 3)
377
42
(377, 224, 224, 3)
(377, 3)
(42, 3)


In [186]:
# Preprocess input for ResNet-50
x_train = keras.applications.resnet50.preprocess_input(x_train)
x_test = keras.applications.resnet50.preprocess_input(x_test)

In [187]:
# Load the ResNet-50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [188]:
# Freeze the weights of the ResNet-50 base model
base_model.trainable = False


In [189]:
# Add a custom classification head
model = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.5),
    keras.layers.Dense(3, activation='softmax')
])



In [190]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [191]:
# Train the model
model.fit(x_train, y_train, epochs=15, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/15
12/12 [==============================] - 6s 208ms/step - loss: 1.0298 - accuracy: 0.7188 - val_loss: 0.6077 - val_accuracy: 0.8810
Epoch 2/15
12/12 [==============================] - 1s 108ms/step - loss: 0.5559 - accuracy: 0.8859 - val_loss: 0.4709 - val_accuracy: 0.9048
Epoch 3/15
12/12 [==============================] - 1s 106ms/step - loss: 0.4320 - accuracy: 0.9231 - val_loss: 0.4298 - val_accuracy: 0.9048
Epoch 4/15
12/12 [==============================] - 1s 106ms/step - loss: 0.3848 - accuracy: 0.9231 - val_loss: 0.3942 - val_accuracy: 0.9048
Epoch 5/15
12/12 [==============================] - 1s 108ms/step - loss: 0.3523 - accuracy: 0.9496 - val_loss: 0.3569 - val_accuracy: 0.9286
Epoch 6/15
12/12 [==============================] - 1s 110ms/step - loss: 0.3071 - accuracy: 0.9682 - val_loss: 0.3692 - val_accuracy: 0.9048
Epoch 7/15
12/12 [==============================] - 1s 107ms/step - loss: 0.2568 - accuracy: 0.9867 - val_loss: 0.3256 - val_accuracy: 0.9286
Epoch 

In [196]:
import tensorflow as tf
# Save the model to a file
model.save('model.h5')

# Later, to load the saved model
model = tf.keras.models.load_model('model.h5')

In [192]:
predictions = model.predict(x_test)

# Convert predictions from probabilities to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Ensure y_test has the same shape and data type as predicted_labels
y_test = np.argmax(y_test, axis=1)

# Calculate accuracy
accuracy = np.mean(predicted_labels == y_test)
print("Accuracy:", accuracy)


2/2 [==============================] - 1s 126ms/step
Accuracy: 0.9761904761904762


In [193]:
predictions2 = model.predict(x_test2)

# Convert predictions from probabilities to class labels
predicted_labels2 = np.argmax(predictions2, axis=1)

# Calculate accuracy
accuracy2 = np.mean(predicted_labels2 == y_test2)
print("Accuracy:", accuracy2)


3/3 [==============================] - 1s 91ms/step
Accuracy: 0.9523809523809523
